In [6]:
from transformers import pipeline, set_seed

generator = pipeline("text-generation", model="gpt2")
set_seed(42)

# Example prompt
prompt = "The impact of artificial intelligence on healthcare"
output = generator(prompt, max_length=150, num_return_sequences=1)

print("✅ GPT-2 Generated Paragraph:\n")
print(output[0]['generated_text'])


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


✅ GPT-2 Generated Paragraph:

The impact of artificial intelligence on healthcare is certainly massive. But that will only ever change if we believe in something called AI – that is, the ability to control individuals who are smarter than us?

In the US, this argument involves an unmitigated moral failure to address the growing potential that AI will replace humans. Despite our best efforts, such a philosophy exists only in theory: this is largely a fantasy, and it exists to serve its theoretical purposes, and to serve the interests of the corporate capitalist class. We are doing our best to avoid the problem, but this is the case only by making an artificial society with many different types – with many different demographics, to varying degrees – that can be used to solve complex problems. As such


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

# 📘 Longer sample paragraph for better learning
text = """
Artificial intelligence is transforming industries by automating tasks, improving efficiency, and enabling data-driven decisions.
Its applications in healthcare, education, and transportation are reshaping the way we live and work.
AI-powered tools are becoming increasingly integrated into daily life, driving innovation and new business models.
"""

# Character mapping
chars = sorted(set(text))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

# Hyperparameters
input_size = len(chars)
hidden_size = 128
seq_length = 40
lr = 0.01

# Data preparation
def create_sequences(text, seq_length):
    X, y = [], []
    for i in range(len(text) - seq_length):
        seq = text[i:i+seq_length]
        target = text[i+1:i+seq_length+1]
        X.append([char_to_ix[ch] for ch in seq])
        y.append([char_to_ix[ch] for ch in target])
    return X, y

X, y = create_sequences(text, seq_length)
X = torch.tensor(X)
y = torch.tensor(y)

# Define the LSTM model
class LSTMTextGen(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(LSTMTextGen, self).__init__()
        self.embed = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.fc = nn.Linear(hidden_size, input_size)

    def forward(self, x, hidden):
        x = self.embed(x)
        out, hidden = self.lstm(x, hidden)
        out = self.fc(out)
        return out, hidden

    def init_hidden(self):
        return (torch.zeros(1, 1, hidden_size), torch.zeros(1, 1, hidden_size))

model = LSTMTextGen(input_size, hidden_size)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Training the model
print("\n⏳ Training LSTM...")
for epoch in range(100):
    for i in range(len(X)):
        input_seq = X[i].unsqueeze(1)
        target_seq = y[i].unsqueeze(1)

        hidden = model.init_hidden()
        optimizer.zero_grad()
        output, _ = model(input_seq, hidden)
        loss = loss_fn(output.view(-1, input_size), target_seq.view(-1))
        loss.backward()
        optimizer.step()
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

# Text generation function
def generate_text(model, start_str, length=500, temperature=1.0):
    model.eval()
    hidden = model.init_hidden()
    input_seq = torch.tensor([[char_to_ix[ch] for ch in start_str]]).T
    result = start_str

    for _ in range(length):
        output, hidden = model(input_seq, hidden)
        last_char_logits = output[-1]
        probs = F.softmax(last_char_logits, dim=1).data.squeeze().pow(1 / temperature)
        probs = probs / probs.sum()
        char_ind = torch.multinomial(probs, 1)[0]
        char = ix_to_char[char_ind.item()]
        result += char
        input_seq = torch.tensor([[char_ind]])

    return result

# Generate paragraph using trained LSTM
print("\n✅ LSTM Generated Paragraph:\n")
print(generate_text(model, "Artificial", length=1000, temperature=1.0))


⏳ Training LSTM...
Epoch 0, Loss: 0.4457
Epoch 10, Loss: 0.0648
Epoch 20, Loss: 0.0287
Epoch 30, Loss: 0.0302
Epoch 40, Loss: 0.0169
Epoch 50, Loss: 0.0367
Epoch 60, Loss: 0.0329
Epoch 70, Loss: 0.0495
Epoch 80, Loss: 0.0092
Epoch 90, Loss: 0.0546

✅ LSTM Generated Paragraph:

Artificial intellligence is transportation and new business modells are becoming increasinablintransportation and new business models.
AI-powered tools are becomincreasin healthcare becoming increasinablthcare becoming increasinablinttrated into daily life, driving innovation and new business models.
AI-powered toools are becomincy, and work.
AI-powered tools are becoming increasinabling data-driven decision,shapingly integly integrated into daily life, driving innovation and new business models.
AI-powered toools areforming increasinabling data-driven decisions.
Its is is is integrated into daily life, driving innovation and new business modells.
AI-powered toools abusfoming increasin healthcare becoming increa